In [19]:
### Read config file
import yaml

def read_config_file(config_file):
    with open(config_file, 'r') as stream:
        try:
            config = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return config

In [20]:
config_file = '/home/shiqi/code/Project2-sensor-case/model_combination_Argos/combined_model_20240805/outputs/experiment_1/config.yaml'
config = read_config_file(config_file)

In [21]:
import sys
sys.path.append('/home/shiqi/code/Project2-sensor-case/model_combination_Argos/utils')
from load_dataset import cut_slices, load_dataset
import numpy as np
import os

def data_preparation(config, data_dir):
    window_size = config['window_size']
    print(f'window_size: {window_size}')
    x_dataset, y_dataset, u_dataset = [], [], []
    # Load data
    for item in os.listdir(data_dir):
        data_file_path = os.path.join(data_dir, item)

        # Check if the file exists before trying to load it
        
        if os.path.exists(data_file_path) and data_file_path.endswith('.npy'):
            
            data_dict = np.load(data_file_path, allow_pickle=True).item()
            x_data, y_data, u_data, _ = load_dataset(data_dict)
            # print(x_data.shape, y_data.shape, u_data.shape)
            x_dataset.append(x_data[1:window_size])
            y_dataset.append(y_data[1:window_size])
            u_dataset.append(u_data[1:window_size])
            # print(f"Loaded data from {data_file_path}")
        else:
            print(f"File not found: {data_file_path}")

    print(x_dataset[0].shape, y_dataset[0].shape, u_dataset[0].shape)
    # Concatenate data
    x_data = np.concatenate(x_dataset, axis=0)
    y_data = np.concatenate(y_dataset, axis=0)
    u_data = np.concatenate(u_dataset, axis=0)
    print(f'x_data shape: {x_data.shape}, y_data shape: {y_data.shape}, u_data shape: {u_data.shape}')

    return x_data, y_data, u_data

In [22]:
import torch
import torch.nn as nn

class StdScalerLayer(nn.Module):
    def __init__(self, mean, std):
        super(StdScalerLayer, self).__init__()
        if not isinstance(mean, torch.Tensor):
            mean = torch.tensor(mean, dtype=torch.float32)
        if not isinstance(std, torch.Tensor):
            std = torch.tensor(std, dtype=torch.float32)
        self.mean = nn.Parameter(mean, requires_grad=False)
        self.std = nn.Parameter(std, requires_grad=False)

    def transform(self, x):
        return (x - self.mean) / self.std
    
    def inverse_transform(self, input):
        return input * self.std + self.mean
    
class Linear_model(torch.nn.Module):
    def __init__(self, state_dim, control_dim):
        super(Linear_model, self).__init__()
        self.A = torch.nn.Parameter(torch.randn(state_dim, state_dim))

        self.B = torch.nn.Parameter(torch.randn(control_dim, state_dim))
    
    def x_dict(self, x):
        ones = torch.ones(x.shape[0], 1).to(x.device)
        return torch.cat((x, ones), dim=1)
    
    def u_dict(self, u):
        return u
    
    def forward(self, x, u):
        x = self.x_dict(x)
        u = self.u_dict(u)
        y = torch.matmul(x, self.A) + torch.matmul(u, self.B)
        return y[:, 1:]
    
class PCALayer(nn.Module):
    def __init__(self, input_dim, output_dim, pca_matrix):
        super(PCALayer, self).__init__()
        self.pca_matrix = pca_matrix
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.transform = nn.Linear(input_dim, output_dim, bias = False)
        self.transform.weight = nn.Parameter(pca_matrix, requires_grad=False)
        self.inverse_transform = nn.Linear(output_dim, input_dim, bias = False)
        self.inverse_transform.weight = nn.Parameter(pca_matrix.T, requires_grad=False)


In [23]:
import torch
device = torch.device('cpu')
from sklearn.decomposition import PCA

# Load data
x_data, y_data, u_data = data_preparation(config, config['train_data_dir'])
x_data = torch.tensor(x_data, dtype=torch.float32).to(device)
y_data = torch.tensor(y_data, dtype=torch.float32).to(device)
u_data = torch.tensor(u_data, dtype=torch.float32).to(device)

x_dim = x_data.shape[-1]
u_dim = u_data.shape[-1]

## PCA
# Standardize data
mean_1 = torch.mean(x_data, dim=0)
std_1 = torch.std(x_data, dim=0)
std_layer_1 = StdScalerLayer(mean_1, std_1)
x_data_scaled = std_layer_1.transform(x_data)

# PCA layer
pca = PCA(n_components=config['pca_dim'])
# Ensure x_data_scaled is converted back to a NumPy array for PCA
pca.fit(x_data_scaled.detach().cpu().numpy())
components = pca.components_
pca_matrix = torch.tensor(components, dtype=torch.float32).to(device)
print(f'PCA matrix shape: {pca_matrix.shape}')
pca_layer = PCALayer(x_dim, config['pca_dim'], pca_matrix)

# Standardize data 2
x_pca = pca_layer.transform(x_data_scaled)
mean_2 = torch.mean(x_pca, dim=0)
std_2 = torch.std(x_pca, dim=0)
std_layer_2 = StdScalerLayer(mean_2, std_2)

# Build pca dataset
x_pca_scaled = std_layer_2.transform(x_pca)
y_data_scaled = std_layer_1.transform(y_data)
y_pca = pca_layer.transform(y_data_scaled)
y_pca_scaled = std_layer_2.transform(y_pca)
mean_u = torch.mean(u_data, dim=0)
std_u = torch.std(u_data, dim=0)
std_layer_u = StdScalerLayer(mean_u, std_u)
u_data_scaled = std_layer_u.transform(u_data)
dataset = [x_pca_scaled, y_pca_scaled, u_data_scaled]
print(f'PCA data shape: {x_pca_scaled.shape}, {y_pca_scaled.shape}, {u_data_scaled.shape}')


window_size: 150
(149, 6957) (149, 6957) (149, 2)
x_data shape: (6258, 6957), y_data shape: (6258, 6957), u_data shape: (6258, 2)
PCA matrix shape: torch.Size([4, 6957])
PCA data shape: torch.Size([6258, 4]), torch.Size([6258, 4]), torch.Size([6258, 2])


In [24]:
import torch

def Build_Hankel_Matrix(y, i, j):
    """
    Construct the Hankel matrix H_y from y(t).
    """
    y_dim = y.shape[0]
    T = y.shape[1]
    H_y = torch.zeros(2*i*y_dim, j)
    if 2 * i + j > T:
        raise ValueError("The given i and j are too large for the given data.")
    for k in range(2 * i):
        H_y[k*y_dim:(k+1)*y_dim, :] = y[:, k:k+j]
    return H_y

def H_past_Matrix(H_y, i, y_dim):
    """
    Construct the past matrix H_{y,past} from the Hankel matrix H_y.
    """
    return H_y[:i*y_dim, :]

def H_future_Matrix(H_y, i, y_dim):
    """
    Construct the future matrix H_{y,future} from the Hankel matrix H_y.
    """
    return H_y[i*y_dim:2*i*y_dim, :]

def H_past_Matrix_plus(H_y, i, y_dim):
    """
    Construct the past matrix H_{y,past}^+ from the Hankel matrix H_y.
    """
    return H_y[:(i+1)*y_dim, :]

def H_future_Matrix_minus(H_y, i, y_dim):
    """
    Construct the future matrix H_{y,future}^- from the Hankel matrix H_y.
    """
    return H_y[(i+1)*y_dim:(2*i+1)*y_dim, :]

In [25]:
import torch

def truncated_svd_pinv(matrix, threshold=1e-2):
    """
    Compute the pseudoinverse of a matrix using truncated SVD to improve numerical stability.
    """
    U, S, V = torch.svd(matrix)
    
    # 只保留大于阈值的奇异值及其对应的U和V
    valid_indices = S > threshold
    S_truncated = S[valid_indices]
    U_truncated = U[:, valid_indices]
    V_truncated = V[:, valid_indices]
    
    # 计算截断后的S_inv
    S_inv = torch.diag(1.0 / S_truncated)
    
    # 计算伪逆矩阵
    return V_truncated @ S_inv @ U_truncated.T

# Define the Orthogonal and Oblique Projections
def Orthogonal_Complement(A):
    """
    Compute the orthogonal complement of matrix A.
    """
    print("Condition number of A @ A.T (regularized):", torch.linalg.cond(A @ A.T + 1e-2 * torch.eye(A.shape[0])))
    # print("A @ A.T:", A @ A.T)
    return torch.eye(A.shape[1]) - A.T @ truncated_svd_pinv(A @ A.T) @ A

def Orthogonal_Projection(A, B):
    """
    Compute the orthogonal projection of the row vectors of matrix A onto the row vectors of matrix B.
    """
    print("Condition number of B @ B.T:", torch.linalg.cond(B @ B.T))
    return A @ B.T @ truncated_svd_pinv(B @ B.T) @ B

def Oblique_Projection(A, B, C):
    """
    Calculate the oblique projection of matrix A onto the subspace spanned by the row vectors of matrix C with the direction of matrix B.
    """
    B_orth = Orthogonal_Complement(B)
    P_ABorth = Orthogonal_Projection(A, B_orth)
    P_CBorth = Orthogonal_Projection(C, B_orth)
    print("Condition number of P_CBorth:", torch.linalg.cond(P_CBorth))
    return P_ABorth @ truncated_svd_pinv(P_CBorth) @ C

In [26]:
def Subspace_Identification(y, u, i, j):
    H_y = Build_Hankel_Matrix(y, i, j)
    H_u = Build_Hankel_Matrix(u, i, j)
    Y_p = H_past_Matrix(H_y, i, y.shape[0])
    Y_f = H_future_Matrix(H_y, i, y.shape[0])
    U_p = H_past_Matrix(H_u, i, u.shape[0])
    U_f = H_future_Matrix(H_u, i, u.shape[0])
    Y_p_plus = H_past_Matrix_plus(H_y, i, y.shape[0])
    Y_f_minus = H_future_Matrix_minus(H_y, i, y.shape[0])
    U_p_plus = H_past_Matrix_plus(H_u, i, u.shape[0])
    U_f_minus = H_future_Matrix_minus(H_u, i, u.shape[0])
    W_p = torch.cat((Y_p, U_p), 0)
    W_p_plus = torch.cat((Y_p_plus, U_p_plus), 0)

    # Calculate the oblique projection
    O_i = Oblique_Projection(Y_f, U_f, W_p)
    O_i_minus = Oblique_Projection(Y_f_minus, U_f_minus, W_p_plus)

    # Calculate the SVD of the weighted oblique projection
    U, S, VT = torch.svd(O_i)
    print(O_i.shape)

    # Determine the order of the system
    threshold = 1e-3
    r = torch.sum(S > threshold).item()
    U_1 = U[:, :r]
    S_1 = torch.diag(S[:r])
    print(S)

    # Determine Gamma_i and Gamma_{i-1}
    Gamma_i = U_1 @ torch.sqrt(S_1)
    print(Gamma_i.shape)
    Gamma_i_minus = Gamma_i[y.shape[0]:, :]

    # Determine X_i and X_{i+1}
    X_i = torch.linalg.pinv(Gamma_i) @ O_i
    X_i_plus = torch.linalg.pinv(Gamma_i_minus) @ O_i_minus

    # Determine the system matrices
    Y_i = H_y[i*y.shape[0]:(i+1)*y.shape[0], :]
    U_i = H_u[i*u.shape[0]:(i+1)*u.shape[0], :]

    X_plus_Y = torch.cat((X_i_plus, Y_i), 0)
    X_U = torch.cat((X_i, U_i), 0)

    System_Matrices = X_plus_Y @ torch.linalg.pinv(X_U)
    A = System_Matrices[:X_i.shape[0], :X_i.shape[0]]
    B = System_Matrices[:X_i.shape[0], X_i.shape[0]:]
    C = System_Matrices[X_i.shape[0]:, :X_i.shape[0]]
    D = System_Matrices[X_i.shape[0]:, X_i.shape[0]:]

    return A, B, C, D


In [27]:
xx = x_pca_scaled[:config['window_size']-1, :].T
uu = u_data_scaled[:config['window_size']-1, :].T
A_pred, B_pred, C_pred, D_pred = Subspace_Identification(xx, uu, 20, 5)

Condition number of A @ A.T (regularized): tensor(6566.5649)
Condition number of B @ B.T: tensor(2.0550e+08)
Condition number of B @ B.T: tensor(2.0550e+08)
Condition number of P_CBorth: tensor(48530760.)
Condition number of A @ A.T (regularized): tensor(6220.4907)
Condition number of B @ B.T: tensor(1.3532e+08)
Condition number of B @ B.T: tensor(1.3532e+08)
Condition number of P_CBorth: tensor(55350480.)
torch.Size([80, 5])
tensor([5.3083e+00, 2.3738e+00, 3.6527e-01, 5.6254e-07, 4.7999e-07])
torch.Size([80, 3])


In [28]:
print(A_pred, B_pred, C_pred, D_pred)

tensor([[ 9.8899e-01,  2.1994e-03, -8.1904e-04],
        [-1.4702e-02,  1.0030e+00, -9.5919e-04],
        [ 1.5825e-02, -3.3876e-03,  1.0011e+00]]) tensor([[ 0.7205, -0.6674],
        [ 0.6617, -0.6398],
        [-3.7559,  3.2415]]) tensor([[-3.7479e-03, -1.7206e-02,  1.4258e-04],
        [-6.1450e-02,  4.5533e-02, -7.1907e-03],
        [-1.9295e-01,  5.4494e-01, -9.5441e-02],
        [-8.4975e-01, -2.0733e-01, -1.8462e-01]]) tensor([[ -7.3423,   6.4658],
        [ -7.0361,   8.3974],
        [ 11.5477, -10.5630],
        [  4.4944,  -3.2356]])


In [29]:
C_pred.shape

torch.Size([4, 3])

In [30]:
def generate_linear_trajectories(y_dataset, u_dataset, matrices, std_layer_1, pca_transformer, std_layer_2, std_layer_u, divice = 'cpu'):
    A, B, C, D = matrices
    x_dim = A.shape[0]
    u_dim = B.shape[0]
    y_data_pca_pred_traj = []
    y_data_pca_traj = []
    y_data_pred_traj = []
    for y_data, u_data in zip(y_dataset, u_dataset):
        y_data = torch.tensor(y_data, dtype=torch.float32).to(device)
        u_data = torch.tensor(u_data, dtype=torch.float32).to(device)
        y_data_scaled = std_layer_1.transform(y_data)
        y_pca = pca_transformer.transform(y_data_scaled)
        y_pca_scaled = std_layer_2.transform(y_pca)
        y_pred = torch.zeros(y_pca_scaled.shape)
        y_pred[0, :] = y_pca_scaled[0, :]

        u_data_scaled = std_layer_u.transform(u_data)
        y0 = y_pca_scaled[0, :].unsqueeze(0)
        x0 = torch.linalg.pinv(C) @ (y0.T - D @ u_data_scaled[0, :].unsqueeze(0).T)
        # print(x0.shape)
        for i in range(1, y_data_scaled.shape[0]):
            x1 = A @ x0 + B @ u_data_scaled[i-1, :].unsqueeze(0).T
            y1 = C @ x1 + D @ u_data_scaled[i-1, :].unsqueeze(0).T
            x0 = x1
            y_pred[i, :] = y1.T
        y_data_pca_traj.append(y_pca_scaled.detach().cpu().numpy())
        y_data_pca_pred_traj.append(y_pred.detach().cpu().numpy())
        y_pca_pred = std_layer_2.inverse_transform(y_pred)
        y_data_pred_scaled = pca_transformer.inverse_transform(y_pca_pred)
        y_data_pred = std_layer_1.inverse_transform(y_data_pred_scaled)
        y_data_pred_traj.append(y_data_pred.detach().cpu().numpy())
    
    return y_data_pred_traj, y_data_pca_pred_traj, y_data_pca_traj

            

In [31]:
# Evaluation data
### Evaluation
def load_evaluation_data(begin, end, data_dir):
    x_dataset = []
    u_dataset = []

    for item in os.listdir(data_dir):
        data_file_path = os.path.join(data_dir, item)

        # Check if the file exists before trying to load it
        if os.path.exists(data_file_path) and item.endswith('.npy'):
            data_dict = np.load(data_file_path, allow_pickle=True).item()
            x_data, _, u_data, _ = load_dataset(data_dict)
            x_dataset.append(x_data[begin:end, :])
            u_dataset.append(u_data[begin:end, :])
        else:
            print(f"File not found: {data_file_path}")
    
    return x_dataset, u_dataset

print(config['begin'], config['end'], config['train_data_dir'])
x_dataset_train, u_dataset_train = load_evaluation_data(config['begin'], config['end'], config['train_data_dir'])
x_dataset_test, u_dataset_test = load_evaluation_data(config['begin'], config['end'], config['test_data_dir'])


1 151 /home/shiqi/code/Project2-sensor-case/model_combination_Argos/data_dir/data_20240621


In [32]:
matrices = [A_pred, B_pred, C_pred, D_pred]
x_data_pred_traj_train, x_data_pca_traj_train, x_data_pca_pred_traj_train = generate_linear_trajectories(y_dataset=x_dataset_train, u_dataset=u_dataset_train, matrices=matrices, std_layer_1=std_layer_1, pca_transformer=pca_layer, std_layer_2=std_layer_2, std_layer_u=std_layer_u)
x_data_pred_traj_test, x_data_pca_traj_test, x_data_pca_pred_traj_test = generate_linear_trajectories(y_dataset=x_dataset_test, u_dataset=u_dataset_test, matrices=matrices, std_layer_1=std_layer_1, pca_transformer=pca_layer, std_layer_2=std_layer_2, std_layer_u=std_layer_u)
np.save('x_data_pred_traj_train_subspace.npy', x_data_pred_traj_train)
np.save('x_data_pred_traj_test_subspace.npy', x_data_pred_traj_test)



In [33]:
def calculate_relative_diff(x_true, x_pred):
    row_norm_diff = np.linalg.norm(x_true - x_pred, axis=1, ord=2)
    max_norm = np.max(np.linalg.norm(x_true, axis=1, ord=2))
    relative_diff = row_norm_diff / max_norm
    return relative_diff

def calculate_mean_relative_diff_set(x_true_traj, x_pred_traj):
    relative_diffs = [calculate_relative_diff(x_true, x_pred) for x_true, x_pred in zip(x_true_traj, x_pred_traj)]
    mean_relative_diffs = np.mean(relative_diffs, axis=0)
    return mean_relative_diffs

def calculate_relative_error(x_true, x_pred):
    row_norm_diff = np.linalg.norm(x_true - x_pred, ord='fro')
    total_norm_true = np.linalg.norm(x_true, ord='fro')
    return row_norm_diff / total_norm_true

def calculate_mean_relative_error_set(x_true_traj, x_pred_traj):
    relative_errors = [calculate_relative_error(x_true, x_pred) for x_true, x_pred in zip(x_true_traj, x_pred_traj)]
    return relative_errors

In [34]:
# # Calculate mean relative error
# mean_relative_errors_train = calculate_mean_relative_error_set(x_dataset_train, x_data_pred_traj_train)
# mean_relative_errors_test = calculate_mean_relative_error_set(x_dataset_test, x_data_pred_traj_test)

# # Calculate mean relative diff
# mean_relative_diffs_train = calculate_mean_relative_diff_set(x_dataset_train, x_data_pred_traj_train)
# mean_relative_diffs_test = calculate_mean_relative_diff_set(x_dataset_test, x_data_pred_traj_test)


In [35]:
x_dataset_train_Ip = [arr[:, -1:] for arr in x_dataset_train]
x_dataset_test_Ip = [arr[:, -1:] for arr in x_dataset_test]
x_dataset_train_Phi = [arr[:, :-1] for arr in x_dataset_train]
x_dataset_test_Phi = [arr[:, :-1] for arr in x_dataset_test]

x_data_pred_traj_train_Ip = [arr[:, -1:] for arr in x_data_pred_traj_train]
x_data_pred_traj_test_Ip = [arr[:, -1:] for arr in x_data_pred_traj_test]
x_data_pred_traj_train_Phi = [arr[:, :-1] for arr in x_data_pred_traj_train]
x_data_pred_traj_test_Phi = [arr[:, :-1] for arr in x_data_pred_traj_test]

np.save(config['save_dir'] + '/x_data_pred_traj_train_subspace_Ip.npy', x_data_pred_traj_train_Ip)
np.save(config['save_dir'] + '/x_data_pred_traj_test_subspace_Ip.npy', x_data_pred_traj_test_Ip)
np.save(config['save_dir'] + '/x_data_pred_traj_train_subspace_Phi.npy', x_data_pred_traj_train_Phi)
np.save(config['save_dir'] + '/x_data_pred_traj_test_subspace_Phi.npy', x_data_pred_traj_test_Phi)

mean_relative_errors_train_Ip = calculate_mean_relative_error_set(x_dataset_train_Ip, x_data_pred_traj_train_Ip)
mean_relative_errors_test_Ip = calculate_mean_relative_error_set(x_dataset_test_Ip, x_data_pred_traj_test_Ip)
mean_relative_errors_train_Phi = calculate_mean_relative_error_set(x_dataset_train_Phi, x_data_pred_traj_train_Phi)
mean_relative_errors_test_Phi = calculate_mean_relative_error_set(x_dataset_test_Phi, x_data_pred_traj_test_Phi)

mean_relative_diffs_train_Ip = calculate_mean_relative_diff_set(x_dataset_train_Ip, x_data_pred_traj_train_Ip)
mean_relative_diffs_test_Ip = calculate_mean_relative_diff_set(x_dataset_test_Ip, x_data_pred_traj_test_Ip)
mean_relative_diffs_train_Phi = calculate_mean_relative_diff_set(x_dataset_train_Phi, x_data_pred_traj_train_Phi)
mean_relative_diffs_test_Phi = calculate_mean_relative_diff_set(x_dataset_test_Phi, x_data_pred_traj_test_Phi)

np.save(config['save_dir'] + '/subspace_mean_relative_errors_train_Ip.npy', mean_relative_errors_train_Ip)
np.save(config['save_dir'] + '/subspace_mean_relative_errors_test_Ip.npy', mean_relative_errors_test_Ip)
np.save(config['save_dir'] + '/subspace_mean_relative_errors_train_Phi.npy', mean_relative_errors_train_Phi)
np.save(config['save_dir'] + '/subspace_mean_relative_errors_test_Phi.npy', mean_relative_errors_test_Phi)
np.save(config['save_dir'] + '/subspace_mean_relative_diffs_train_Ip.npy', mean_relative_diffs_train_Ip)
np.save(config['save_dir'] + '/subspace_mean_relative_diffs_test_Ip.npy', mean_relative_diffs_test_Ip)
np.save(config['save_dir'] + '/subspace_mean_relative_diffs_train_Phi.npy', mean_relative_diffs_train_Phi)
np.save(config['save_dir'] + '/subspace_mean_relative_diffs_test_Phi.npy', mean_relative_diffs_test_Phi)


In [36]:
# np.save('subspace_mean_relative_errors_train.npy', mean_relative_errors_train)
# np.save('subspace_mean_relative_errors_test.npy', mean_relative_errors_test)
# np.save('subspace_mean_relative_diffs_train.npy', mean_relative_diffs_train)
# np.save('subspace_mean_relative_diffs_test.npy', mean_relative_diffs_test)


In [37]:
# import numpy as np
# import matplotlib.pyplot as plt

# # 随机挑选10个样本的行索引
# num_samples = 2
# indices = np.random.choice(len(x_data_pca_traj_train), num_samples, replace=False)
# print(indices)

# # 创建4列10行的子图
# fig, axs = plt.subplots(num_samples, 4, figsize=(16, 4 * num_samples))

# # 循环处理每一个随机选取的样本
# for i, idx in enumerate(indices):
#     # 获取x_data_pred_traj_train和x_data_pca_traj_train的第idx个样本
#     pred_traj = x_data_pred_traj_train[idx]  # 第idx行的数据
#     pca_traj = x_data_pca_traj_train[idx]    # 对应的PCA
#     print(i)

#     # 绘制该样本的不同维度
#     for j in range(4):
#         axs[i, j].plot(pred_traj[:,j], label='Pred Traj')
#         axs[i, j].plot(pca_traj[:, j], label='PCA Traj')
#         axs[i, j].set_title(f'Sample {idx} - Dimension {j+1}')
#         axs[i, j].legend()
#         axs[i, j].grid(True)

# # 设置整体布局和展示
# plt.tight_layout()
# plt.show()


In [38]:
# calculate_relative_error(x_data_pca_traj_train[4], x_data_pca_pred_traj_train[4])

In [39]:
# import numpy as np
# import matplotlib.pyplot as plt

# # 随机挑选10个样本的行索引
# num_samples = 10
# indices = np.random.choice(len(x_data_pca_traj_train), num_samples, replace=False)
# print(indices)

# # 创建4列10行的子图
# fig, axs = plt.subplots(num_samples, 4, figsize=(16, 4 * num_samples))

# # 循环处理每一个随机选取的样本
# for i, idx in enumerate(indices):
#     # 获取x_data_pred_traj_train和x_data_pca_traj_train的第idx个样本
#     pred_traj = x_dataset_train[idx]  # 第idx行的数据
#     pca_traj = x_data_pred_traj_train[idx]    # 对应的PCA
#     print(i)

#     # 绘制该样本的不同维度
#     for j in range(4):
#         axs[i, j].plot(pred_traj[:,j], label='Pred Traj')
#         axs[i, j].plot(pca_traj[:, j], label='PCA Traj')
#         axs[i, j].set_title(f'Sample {idx} - Dimension {j+1}')
#         axs[i, j].legend()
#         axs[i, j].grid(True)

# # 设置整体布局和展示
# plt.tight_layout()
# plt.show()
